### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-09-05


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-09-05
Yesterday: 2025-09-04


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}','price':'{:.2f}',
    'beta':'{:,.2f}','change':'{:.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}','volume':'{:,.0f}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-09-04'
ORDER BY a.name



(16, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-09-04' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df.style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
AIMIRT,2025-09-02,2025-09-04,9.85,9.75,"1,279,280",9.85,9.60,B,SET,-0.10,-1.02%
CPF,2025-09-03,2025-09-04,22.40,21.80,"659,558,256",24.50,21.80,B,SET50,-0.60,-2.68%
GVREIT,2025-09-03,2025-09-04,6.20,6.25,"8,815,456",6.25,6.15,B,SET,0.05,0.81%
JMART,2025-09-04,2025-09-04,9.05,9.00,"79,992,472",9.05,8.25,B,SET50,-0.05,-0.55%
JMT,2025-09-04,2025-09-04,12.10,11.80,"38,869,624",12.10,11.60,B,SET50,-0.30,-2.48%
ORI,2025-09-01,2025-09-04,2.18,2.24,"215,661,432",2.32,2.16,B,SET100,0.06,2.75%
PTT,2025-09-04,2025-09-04,32.00,31.75,"941,613,172",32.25,31.00,B,SET50,-0.25,-0.78%
RCL,2025-09-04,2025-09-04,27.00,27.25,"30,630,752",28.25,27.00,B,SET100,0.25,0.93%
TFFIF,2025-09-04,2025-09-04,5.85,5.90,"119,859,864",6.10,5.85,B,SET,0.05,0.85%
WHAIR,2025-08-28,2025-09-04,5.55,5.45,"21,582,544",5.60,5.45,B,SET,-0.10,-1.80%


In [20]:
df.shape

(16, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
AIMIRT,2025-09-02,2025-09-04,9.85,9.75,"1,279,280",9.85,9.60,-1.02%,B
CPF,2025-09-03,2025-09-04,22.40,21.80,"659,558,256",24.50,21.80,-2.68%,B
GVREIT,2025-09-03,2025-09-04,6.20,6.25,"8,815,456",6.25,6.15,0.81%,B
JMART,2025-09-04,2025-09-04,9.05,9.00,"79,992,472",9.05,8.25,-0.55%,B
JMT,2025-09-04,2025-09-04,12.10,11.80,"38,869,624",12.10,11.60,-2.48%,B
ORI,2025-09-01,2025-09-04,2.18,2.24,"215,661,432",2.32,2.16,2.75%,B
PTT,2025-09-04,2025-09-04,32.00,31.75,"941,613,172",32.25,31.00,-0.78%,B
RCL,2025-09-04,2025-09-04,27.00,27.25,"30,630,752",28.25,27.00,0.93%,B
TFFIF,2025-09-04,2025-09-04,5.85,5.90,"119,859,864",6.10,5.85,0.85%,B
WHAIR,2025-08-28,2025-09-04,5.55,5.45,"21,582,544",5.60,5.45,-1.80%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(16, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(16, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu].style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
AIMIRT,2025-09-02,2025-09-04,9.85,9.75,"1,279,280",9.85,9.60,-1.02%,B,9.75,0.00,"150,055",2025-09-05
CPF,2025-09-03,2025-09-04,22.40,21.80,"659,558,256",24.50,21.80,-2.68%,B,22.50,0.70,"34,448,126",2025-09-05
GVREIT,2025-09-03,2025-09-04,6.20,6.25,"8,815,456",6.25,6.15,0.81%,B,6.30,0.05,"439,091",2025-09-05
JMART,2025-09-04,2025-09-04,9.05,9.00,"79,992,472",9.05,8.25,-0.55%,B,9.45,0.45,"30,472,352",2025-09-05
JMT,2025-09-04,2025-09-04,12.10,11.80,"38,869,624",12.10,11.60,-2.48%,B,12.20,0.40,"25,500,622",2025-09-05
ORI,2025-09-01,2025-09-04,2.18,2.24,"215,661,432",2.32,2.16,2.75%,B,2.42,0.18,"22,585,406",2025-09-05
PTT,2025-09-04,2025-09-04,32.00,31.75,"941,613,172",32.25,31.00,-0.78%,B,31.50,-0.25,"56,140,730",2025-09-05
RCL,2025-09-04,2025-09-04,27.00,27.25,"30,630,752",28.25,27.00,0.93%,B,27.25,0.00,"1,318,568",2025-09-05
TFFIF,2025-09-04,2025-09-04,5.85,5.90,"119,859,864",6.10,5.85,0.85%,B,5.90,0.00,"1,293,221",2025-09-05
WHAIR,2025-08-28,2025-09-04,5.55,5.45,"21,582,544",5.60,5.45,-1.80%,B,5.50,0.05,"190,311",2025-09-05


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AIMIRT   2025-09-02 2025-09-05   9.85   9.75  -1.02%   1,279,280   9.85   9.60 B Update
CPF      2025-09-05 2025-09-05  21.80  22.50   3.21% 659,558,256  24.50  21.80 B Insert
DIF      2025-08-26 2025-09-05   8.20   8.55   4.27% 463,957,936   8.55   8.15 S Update
GVREIT   2025-09-03 2025-09-05   6.20   6.30   1.61%   8,815,456   6.30   6.15 B Update
IVL      2025-09-05 2025-09-05  21.50  22.10   2.79% 208,637,968  23.40  21.50 S Insert
JMART    2025-09-05 2025-09-05   9.00   9.45    5.0%  79,992,472   9.45   8.25 B Insert
JMT      2025-09-05 2025-09-05  11.80  12.20   3.39%  38,869,624  12.20  11.60 B Insert
MCS      2025-09-03 2025-09-05   8.25   8.50   3.03%   2,416,752   8.50   8.15 S Update
NER      2025-08-29 2025-09-05   4.26   4.46   4.69%  70,962,128   4.46   4.26 S Update
ORI      2025-09-01 2025-09-05  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [60]:
sql = """
    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '{}' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name
""".format(today)
print(sql)


    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '2025-09-05' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name



In [62]:
df = pd.read_sql(sql, conlite)
df.shape

(16, 10)

In [64]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
AIMIRT,2025-09-02,2025-09-05,9.85,9.75,"2,558,560",9.85,9.60,B,SET,-0.10,-1.02%
CPF,2025-09-05,2025-09-05,21.80,22.50,"659,558,256",24.50,21.80,B,SET50,0.70,3.21%
GVREIT,2025-09-03,2025-09-05,6.20,6.30,"17,630,912",6.30,6.15,B,SET,0.10,1.61%
JMART,2025-09-05,2025-09-05,9.00,9.45,"79,992,472",9.45,8.25,B,SET50,0.45,5.00%
JMT,2025-09-05,2025-09-05,11.80,12.20,"38,869,624",12.20,11.60,B,SET50,0.40,3.39%
ORI,2025-09-01,2025-09-05,2.18,2.42,"431,322,864",2.42,2.16,B,SET100,0.24,11.01%
PTT,2025-09-04,2025-09-05,32.00,31.50,"1,883,226,344",32.25,31.00,B,SET50,-0.50,-1.56%
RCL,2025-09-04,2025-09-05,27.00,27.25,"61,261,504",28.25,27.00,B,SET100,0.25,0.93%
TFFIF,2025-09-04,2025-09-05,5.85,5.90,"239,719,728",6.10,5.85,B,SET,0.05,0.85%
WHAIR,2025-09-05,2025-09-05,5.45,5.50,"21,582,544",5.60,5.45,B,SET,0.05,0.92%


In [66]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [68]:
df_sort.query('percent <= -5.00').style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [70]:
df_sort.query('percent >= 5.00').style.format(format_dict).hide(axis="index")

name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
JMART,2025-09-05,2025-09-05,9.00,9.45,"79,992,472",9.45,8.25,B,SET50,0.45,5.00%
PTG,2025-09-02,2025-09-05,7.65,8.45,"153,989,920",8.45,7.60,S,SET100,0.80,10.46%
ORI,2025-09-01,2025-09-05,2.18,2.42,"431,322,864",2.42,2.16,B,SET100,0.24,11.01%


In [72]:
conlite.close()

In [74]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-09-05 22:00:15
